In [167]:
from functools import partial
from pyscf import gto, scf, cc

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1
nH = 4
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", verbose=4)
mf = scf.RHF(mol).density_fit()
mf.kernel()

mf2 = scf.RHF(mol)#.density_fit()
mf2.kernel()

# cc
mycc = cc.CCSD(mf)
mycc.kernel()
et = mycc.ccsd_t()

print(f"ccsd energy is {mycc.e_tot}")
print(f"ccsd_t energy is {mycc.e_tot+et}")

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Wed Jul 16 15:38:38 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

ecp = {}
CPU time:       172.86


******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = False
chkfile to save SCF result = /tmp/tmpc4x76jie
max_memory 4000 MB (current use 985 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 4000
Default auxbasis cc-pvdz-jkfit is used for H ccpvdz
init E= -1.65017133581905
  HOMO = -0.321825878209554  LUMO = 0.0325650635953273
cycle= 1 E= -2.16142847244588  delta_E= -0.511  |g|= 0.0907  |ddm|= 1.02
  HOMO = -0.440368877568953  LUMO = 0.0824150240082916
cycle= 2 E= -2.16609297252703  delta_E= -0.00466  |g|= 0.0226  |ddm|= 0.28
  HOMO = -0.434670863329405  LUMO = 0.095499477889071
cycle= 3 E= -2.1664755825947  delta_E= -0.0

In [168]:
import numpy as np
from functools import reduce
_fdot = np.dot
fdot = lambda *args: reduce(_fdot, args)

In [138]:
from pyscf import mcscf
mo_coeff = mf.mo_coeff
nfrozen = 0
norb_act = mol.nao - nfrozen
nelec_act = mol.nelectron - 2 * nfrozen
mc = mcscf.CASSCF(mf,norb_act,nelec_act) 
h1e, enuc = mc.get_h1eff()
print('h0:',enuc)
print('df-scf:',mf.energy_nuc(),mf.e_tot)
print('scf:',mf2.energy_nuc(),mf2.e_tot)

h0: 2.29310124732
df-scf: 2.29310124732 -2.1150449906805138
scf: 2.29310124732 -2.1124606989145667


In [169]:
from pyscf import ao2mo
eri_pyscf = ao2mo.kernel(mol,mf2.mo_coeff)

In [170]:
from ad_afqmc.lno.cc import LNOCCSD
from pyscf.lib import logger
import sys
from ad_afqmc.lno.base import lno
from ad_afqmc.lno_ccsd import lno_ccsd
import numpy as np
from functools import reduce
log = logger.Logger(sys.stdout, 6)

options = {
    "n_eql": 4,
    "n_ene_blocks": 1,
    "n_sr_blocks": 20,
    "n_blocks": 20,
    "n_walkers": 5,
    "seed": 98,
    "trial": "cisd",
    "walker_type": "rhf",
    "dt":0.005,
    "ene0": 0,
}

thresh = 1e-6
chol_cut = 1e-6
frozen = 0
run_frg_list = None
lo_type = 'pm'
no_type = 'ie' # cim
frag_lolist = '1o'

_fdot = np.dot
fdot = lambda *args: reduce(_fdot, args)

if isinstance(thresh, float):
    thresh_occ = thresh*10
    thresh_vir = thresh
else:
    thresh_occ, thresh_vir = thresh

lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = lo_type
lno_cc.no_type = no_type
lno_cc.frag_lolist = frag_lolist
lno_cc.ccsd_t = True
lno_cc.force_outcore_ao2mo = True

frag_atmlist = lno_cc.frag_atmlist
s1e = lno_cc._scf.get_ovlp()


log.info('no_type = %s', no_type)

# LO construction
orbloc = lno_cc.get_lo(lo_type=lo_type) # localized active occ orbitals
orbactocc = lno_cc.split_mo()[1] # non-localized active occ
m = fdot(orbloc.T, s1e, orbactocc)
lospanerr = abs(fdot(m.T, m) - np.eye(m.shape[1])).max()
if lospanerr > 1e-10:
    log.error('LOs do not fully span the occupied space! '
                'Max|<occ|LO><LO|occ>| = %e', lospanerr)
    raise RuntimeError

# check 2: Span(LO) == Span(occ)
occspanerr = abs(fdot(m, m.T) - np.eye(m.shape[0])).max()
if occspanerr < 1e-10:
    log.info('LOs span exactly the occupied space.')
    if no_type not in ['ir','ie']:
        log.error('"no_type" must be "ir" or "ie".')
        raise ValueError
else:
    log.info('LOs span occupied space plus some virtual space.')

# LO assignment to fragments

if frag_lolist == '1o':
    log.info('Using single-LO fragment') # this is what we use, every active local occ labels a fragment
    frag_lolist = [[i] for i in range(orbloc.shape[1])]
else: print('Only support single LO fragment!')
nfrag = len(frag_lolist)
frag_nonvlist = lno_cc.frag_nonvlist

# dump info
log.info('nfrag = %d  nlo = %d', nfrag, orbloc.shape[1])
log.info('frag_atmlist = %s', frag_atmlist)
log.info('frag_lolist = %s', frag_lolist)
log.info('frag_nonvlist = %s', frag_nonvlist)

if not (no_type[0] in 'rei' and no_type[1] in 'rei'):
    log.warn('Input no_type "%s" is invalid.', no_type)
    raise ValueError

if frag_nonvlist is None: frag_nonvlist = [[None,None]] * nfrag

eris = lno_cc.ao2mo()
frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [thresh_occ,thresh_vir]
print(f'lno thresh {thresh_pno}')

if run_frg_list is None:
    run_frg_list = range(nfrag)

from jax import random
seeds = random.randint(random.PRNGKey(options["seed"]),
                    shape=(nfrag,), minval=0, maxval=100000*nfrag)

# non df mean-field object
mf2 = scf.RHF(mf.mol)
mf2.kernel()
#e0 = mf2.e_tot

# for ifrag in run_frg_list:
for ifrag in [0]:
    print(f'########### running fragment {ifrag+1} ##########')
    #if(len(lno_cc.runfrags)>0):
    #    if(ifrag not in lno_cc.runfrags):frag_res[ifrag] = (0,0,0)
    fraglo = frag_lolist[ifrag]
    orbfragloc = orbloc[:,fraglo] # the specific local active occ
    frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    THRESH_INTERNAL = 1e-10
    frzfrag, orbfrag, can_orbfrag = lno.make_fpno1(lno_cc, eris, orbfragloc, no_type,
                                                THRESH_INTERNAL, thresh_pno,
                                                frozen_mask=frozen_mask,
                                                frag_target_nocc=frag_target_nocc,
                                                frag_target_nvir=frag_target_nvir,
                                                canonicalize=False)

    ecorr_ccsd,_,t1,t2 = lno_ccsd.cc_impurity_solve(
        mf,orbfrag,orbfragloc,frozen=frzfrag,eris=eris,log=log)
    print(f'# lno-ccsd correlation energy is: {ecorr_ccsd}')

    maskocc = mf.mo_occ>1e-10
    nmo = mf.mo_occ.size

    if frzfrag is None:
        frzfrag = 0
    elif isinstance(frzfrag, (list,tuple,np.ndarray)) and len(frzfrag) == 0:
        frzfrag = 0

    if isinstance(frzfrag, (int,np.integer)):
        maskact = np.hstack([np.zeros(frzfrag,dtype=bool),
                                np.ones(nmo-frzfrag,dtype=bool)])
    elif isinstance(frzfrag, (list,tuple,np.ndarray)):
        maskact = np.array([i not in frzfrag for i in range(nmo)])
    else:
        raise RuntimeError

    orbfrzocc = orbfrag[:,~maskact& maskocc]
    orbactocc = orbfrag[:, maskact& maskocc]
    orbactvir = orbfrag[:, maskact&~maskocc]
    orbfrzvir = orbfrag[:,~maskact&~maskocc]
    _, nactocc, nactvir, _ = [orb.shape[1]
                                            for orb in [orbfrzocc,orbactocc,
                                                        orbactvir,orbfrzvir]]
    s1e = mf.get_ovlp() if eris is None else eris.s1e
    # overlap between the lo and each active NO occ in its fragment
    prjlo = fdot(orbfragloc.T, s1e, orbactocc)
    print('# lo projection on its active NO occ fragment subspace', prjlo)

    options["seed"] = seeds[ifrag]
    lno_ccsd.prep_lno_amp_chol_file(
        mf2,orbfrag,options,
        norb_act=(nactocc+nactvir),nelec_act=nactocc*2,
        prjlo=prjlo,norb_frozen=frzfrag,
        t1=t1,t2=t2,chol_cut=chol_cut
                )
    
    

no_type = ie
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 2  nlo = 2
frag_atmlist = None
frag_lolist = [[0], [1]]
frag_nonvlist = None
Lov is saved to /tmp/azhckt0z
lno thresh [9.999999999999999e-06, 1e-06]


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmp6_rok7dk
max_memory 4000 MB (current use 991 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -1.65016878193605
  HOMO = -0.321825357345338  LUMO = 0.0325662981846424
cycle= 1 E= -2.16142775229274  delta_E= -0.511  |g|= 0.0907  |ddm|= 1.02
  HOMO = -0.44036730832241  LUMO = 0.0824076478034913
cycle= 2 E= -2.16609132033986  delta_E= -0.00466  |g|= 0.0226  |ddm|= 0.2

In [171]:
from ad_afqmc import config, wavefunctions
from jax import numpy as jnp
from jax import random
config.setup_jax()
MPI = config.setup_comm()

ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
    lno_ccsd.prep_lnoccsd_afqmc())

comm = MPI.COMM_WORLD
rank = comm.Get_rank()  # Process rank
size = comm.Get_size() 

seed = options["seed"]
propagator = prop
init_walkers = None
ham_data = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                                )._build_measurement_intermediates(ham_data, wave_data)
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, propagator, trial, wave_data)

prop_data = propagator.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 20
# nelec: (2, 2)
# nchol: 127
#
# n_eql: 4
# n_ene_blocks: 1
# n_sr_blocks: 20
# n_blocks: 20
# n_walkers: 5
# seed: 178631
# trial: cisd
# walker_type: rhf
# dt: 0.005
# ene0: 0
# n_exp_terms: 6
# n_prop_steps: 50
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# free_projection: False
# n_batch: 1
# use_gpu: False
#


In [172]:
walker = prop_data['walkers'][0]
e_init = jnp.real(trial._calc_energy_restricted(walker,ham_data,wave_data))
print('afqmc/ccsd init energy:',e_init)
print('mean-field energy:',mf2.e_tot)
e_corr_init = e_init - mf2.e_tot
print('afqmc/ccsd init correlation energy:',e_corr_init)
print('df-ccsd init correlation energy:',mycc.e_tot-mf.e_tot)

afqmc/ccsd init energy: -2.2529811155354986
mean-field energy: -2.166491465750002
afqmc/ccsd init correlation energy: -0.08648964978549678
df-ccsd init correlation energy: -0.0864760918683265


In [79]:
ham_data['chol'].shape

(11, 16)

In [173]:
lno_ccsd._tot_ccsd_cr(walker,ham_data,wave_data,trial,1e-5)

Array(-0.08648967, dtype=float64)

In [3]:
from ad_afqmc import wavefunctions, config
from jax import numpy as jnp
from jax import random
from jax import config
config.update("jax_enable_x64", True)
mo_coeff = mf.mo_coeff
nfrozen = 0
norb_act = mol.nao - nfrozen
nelec_act = mol.nelectron - 2 * nfrozen
t1 = mycc.t1
t2 = mycc.t2

options = {
    "n_eql": 4,
    "n_ene_blocks": 1,
    "n_sr_blocks": 20,
    "n_blocks": 20,
    "n_walkers": 5,
    "seed": 98,
    "trial": "cisd",
    "walker_type": "rhf",
    "dt":0.005,
    "ene0": 0,
}

from ad_afqmc.lno_ccsd import lno_ccsd
lno_ccsd.prep_lno_amp_chol_file(mf,mo_coeff,options,norb_act=norb_act,nelec_act=nelec_act,
                                prjlo=[],norb_frozen=nfrozen,t1=t1,t2=t2,
                                chol_cut=1e-7,
                                option_file='options.bin',
                                mo_file="mo_coeff.npz",
                                amp_file="amplitudes.npz",
                                chol_file="FCIDUMP_chol"
                                )

# Generating Cholesky Integrals
# frozen orbitals: []
# local active orbitals: [0 1 2 3]
# local active space size: 4
# loc_eris shape: (16, 16)
# chol shape: (10, 16)
# Finished calculating Cholesky integrals

# Size of the correlation space
# Number of electrons: (2, 2)
# Number of basis functions: 4
# Number of Cholesky vectors: 10



In [5]:
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
    lno_ccsd.prep_lnoccsd_afqmc())

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 4
# nelec: (2, 2)
# nchol: 10
#
# n_eql: 4
# n_ene_blocks: 1
# n_sr_blocks: 20
# n_blocks: 20
# n_walkers: 5
# seed: 98
# trial: cisd
# walker_type: rhf
# dt: 0.005
# ene0: 0
# n_exp_terms: 6
# n_prop_steps: 50
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# free_projection: False
# n_batch: 1
# use_gpu: False
#


In [143]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()

comm = MPI.COMM_WORLD
rank = comm.Get_rank()  # Process rank
size = comm.Get_size() 

seed = options["seed"]
propagator = prop
init_walkers = None
ham_data = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                                )._build_measurement_intermediates(ham_data, wave_data)
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, propagator, trial, wave_data)

prop_data = propagator.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [147]:
walker = prop_data['walkers'][0]
e_init = jnp.real(trial._calc_energy_restricted(walker,ham_data,wave_data))
print(e_init)
e_corr_init = e_init - mf2.e_tot
print(e_corr_init)
print(mycc.e_tot-mf.e_tot)

-2.1812025822009646
-0.06874188328639974
-0.06902628862909266


In [152]:
print(f'afqmc/ccsd e_corr: {jnp.real(trial._calc_energy_restricted(walker,ham_data,wave_data))-mf2.e_tot:.9f}')
print(f'tot_lno_afqmc/ccsd e_corr: {lno_ccsd._tot_ccsd_cr(walker,ham_data,wave_data,trial,1e-5):.9f}')

afqmc/ccsd e_corr: -0.068741883
tot_lno_afqmc/ccsd e_corr: -0.068816867


In [165]:
import jax
from jax import lax, vmap, jvp, random, jit
def _tot_ccsd_cr(
    walker: jax.Array,
    ham_data: dict,
    wave_data: dict,
    trial: wavefunctions,
    eps :float = 1e-5
):
    """Calculates local energy using AD and finite difference for the two body term"""

    norb = trial.norb
    chol = ham_data["chol"].reshape(-1, norb, norb)
    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    # v1 the one-body energy from the reordering of the 
    # two-body operators into non-normal ordered form
    v0 = 0.5 * jnp.einsum("gik,gjk->ij",chol,chol,optimize="optimal")
    h1_mod = h1 - v0
    ccsd_olp = trial._calc_overlap_restricted(walker, wave_data)

    # zero body
    h0_E0 = ham_data["h0"]-ham_data["E0"]
    print(h0_E0)
    mod_olp = lno_ccsd._modified_ccsd_olp_restricted(walker,wave_data)

    # one body
    x = 0.0
    f1 = lambda a: lno_ccsd._olp_exp1(a,h1_mod,walker,wave_data)
    _, d_overlap = jvp(f1, [x], [1.0])

    # two body
    # carry: [eps, walker, wave_data]
    def scanned_fun(carry, x):
        eps, walker, wave_data = carry
        return carry, lno_ccsd._olp_exp2(eps,x,walker,wave_data)

    _, overlap_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, overlap_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, overlap_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d_2_overlap = (overlap_p - 2.0 * overlap_0 + overlap_m) / eps / eps

    ccsd_cr0 = jnp.real(h0_E0*mod_olp/ccsd_olp)
    ccsd_cr1 = jnp.real(d_overlap/ccsd_olp)
    ccsd_cr2 = jnp.real(0.5*jnp.sum(d_2_overlap)/ccsd_olp)

    ccsd_cr = jnp.real((h0_E0*mod_olp+d_overlap+jnp.sum(d_2_overlap)/2.0)/ccsd_olp)

    return ccsd_cr0,ccsd_cr1,ccsd_cr2,ccsd_cr

In [155]:
def _afqmc_ccsd_energy(
        walker: jax.Array, ham_data: dict, wave_data: dict, trial
) -> complex:
    ci1, ci2 = wave_data["ci1"], wave_data["ci2"]
    norb = trial.norb
    nocc = trial.nelec[0]
    green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    green_occ = green[:, nocc:].copy()
    greenp = jnp.vstack((green_occ, -jnp.eye(norb - nocc)))

    chol = ham_data["chol"].reshape(-1, norb, norb)
    rot_chol = ham_data["rot_chol"]
    #rot_chol = chol[:, : nocc, :]
    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    hg = jnp.einsum("pj,pj->", h1[:nocc, :], green)

    # 0 body energy
    e0 = jnp.real(ham_data["h0"])

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # single excitations
    ci1g = jnp.einsum("pt,pt->", ci1, green_occ, optimize="optimal")
    e1_1_1 = 4 * ci1g * hg
    gpci1 = greenp @ ci1.T
    ci1_green = gpci1 @ green
    e1_1_2 = -2 * jnp.einsum("ij,ij->", h1, ci1_green, optimize="optimal")
    e1_1 = e1_1_1 + e1_1_2

    # double excitations
    ci2g_c = jnp.einsum("ptqu,pt->qu", ci2, green_occ)
    ci2g_e = jnp.einsum("ptqu,pu->qt", ci2, green_occ)
    ci2_green_c = (greenp @ ci2g_c.T) @ green
    ci2_green_e = (greenp @ ci2g_e.T) @ green
    ci2_green = 2 * ci2_green_c - ci2_green_e
    ci2g = 2 * ci2g_c - ci2g_e
    gci2g = jnp.einsum("qu,qu->", ci2g, green_occ, optimize="optimal")
    e1_2_1 = 2 * hg * gci2g
    e1_2_2 = -2 * jnp.einsum("ij,ij->", h1, ci2_green, optimize="optimal")
    e1_2 = e1_2_1 + e1_2_2
    # e1 = e1_0 + e1_1 + e1_2
    ecorr1 = e1_1 + e1_2

    # two body energy
    # ref
    lg = jnp.einsum("gpj,pj->g", rot_chol, green, optimize="optimal")
    # lg1 = jnp.einsum("gpj,pk->gjk", rot_chol, green, optimize="optimal")
    lg1 = jnp.einsum("gpj,qj->gpq", rot_chol, green, optimize="optimal")
    e2_0_1 = 2 * lg @ lg
    e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg1))
    e2_0 = e2_0_1 + e2_0_2

    # single excitations
    e2_1_1 = 2 * e2_0 * ci1g
    lci1g = jnp.einsum("gij,ij->g", chol, ci1_green, optimize="optimal")
    e2_1_2 = -2 * (lci1g @ lg)
    # lci1g1 = jnp.einsum("gij,jk->gik", chol, ci1_green, optimize="optimal")
    # glgpci1 = jnp.einsum(("gpi,iq->gpq"), gl, gpci1, optimize="optimal")
    ci1g1 = ci1 @ green_occ.T
    # e2_1_3 = jnp.einsum("gpq,gpq->", glgpci1, lg1, optimize="optimal")
    e2_1_3_1 = jnp.einsum("gpq,gqr,rp->", lg1, lg1, ci1g1, optimize="optimal")
    lci1g = jnp.einsum("gip,qi->gpq", ham_data["lci1"], green, optimize="optimal")
    e2_1_3_2 = -jnp.einsum("gpq,gqp->", lci1g, lg1, optimize="optimal")
    e2_1_3 = e2_1_3_1 + e2_1_3_2
    e2_1 = e2_1_1 + 2 * (e2_1_2 + e2_1_3)

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g = jnp.einsum("gij,ij->g", chol, ci2_green, optimize="optimal")
    e2_2_2_1 = -lci2g @ lg

    # lci2g1 = jnp.einsum("gij,jk->gik", chol, ci2_green, optimize="optimal")
    # lci2_green = jnp.einsum("gpi,ji->gpj", rot_chol, ci2_green, optimize="optimal")
    # e2_2_2_2 = 0.5 * jnp.einsum("gpi,gpi->", gl, lci2_green, optimize="optimal")
    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = jnp.einsum("pj,ji->pi", green, chol_i, optimize="optimal")
        lci2_green_i = jnp.einsum(
            "pi,ji->pj", rot_chol_i, ci2_green, optimize="optimal"
        )
        carry[0] += 0.5 * jnp.einsum(
            "pi,pi->", gl_i, lci2_green_i, optimize="optimal"
        )
        glgp_i = jnp.einsum("pi,it->pt", gl_i, greenp, optimize="optimal").astype(
            jnp.complex64
        )
        l2ci2_1 = jnp.einsum(
            "pt,qu,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_2 = jnp.einsum(
            "pu,qt,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        carry[1] += 2 * l2ci2_1 - l2ci2_2
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol, rot_chol))
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)

    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    # e2 = e2_0 + e2_1 + e2_2
    e1_0 = jnp.real(e1_0)
    e2_0 = jnp.real(e2_0)
    e_mf = jnp.real(e0+e1_0+e2_0)
    ecorr2 = e2_1 + e2_2

    # overlap
    overlap_1 = 2 * ci1g  # jnp.einsum("ia,ia", ci1, green_occ)
    overlap_2 = gci2g
    overlap = 1.0 + overlap_1 + overlap_2

    ecorr1 = jnp.real(ecorr1/overlap)
    ecorr2 = jnp.real(ecorr2/overlap)
    ecorr = jnp.real(ecorr1 + ecorr2)

    return e0, e1_0, e2_0, e_mf, ecorr1, ecorr2, ecorr #(e1 + e2) / overlap + e0

In [174]:
e0, e1_0, e2_0, e_mf, ecorr1, ecorr2, ecorr \
    = _afqmc_ccsd_energy(walker,ham_data,wave_data,trial)
print('afqmc/ccsd mean-field energy', e_mf)
print('scf mean-field energy:', mf2.e_tot)
print('afqmc/ccsd sum single-double excitation terms:',ecorr)
print('tot afqmc/ccsd energy:',ecorr+e_mf)
print('tot_lno_afqmc/ccsd correlation energy:',lno_ccsd._tot_ccsd_cr(walker,ham_data,wave_data,trial,1e-5))

afqmc/ccsd mean-field energy -2.1664914655592726
scf mean-field energy: -2.166491465750002
afqmc/ccsd sum single-double excitation terms: -0.086489649976226
tot afqmc/ccsd energy: -2.2529811155354986
tot_lno_afqmc/ccsd correlation energy: -0.08648967204852878


In [119]:
jnp.real(trial._calc_energy_restricted(walker,ham_data,wave_data))

Array(-2.18120258, dtype=float64)

In [127]:
print(mf2.energy_nuc(),mf2.energy_elec()[0],mf2.e_tot)

2.29310124732 -4.405561946234566 -2.112460698914566


In [106]:
e_init = jnp.real(trial._calc_energy_restricted(walker,ham_data,wave_data))
print(e_init)

-2.1812025822009664


In [133]:
hf_cr = lno_ccsd._tot_hf_cr(ham_data['rot_h1'],ham_data['rot_chol'],walker,trial,wave_data)
ccsd_cr0,ccsd_cr1,ccsd_cr2,ccsd_cr = _tot_ccsd_cr(walker,ham_data,wave_data,trial,1e-5)
print(hf_cr)
print(ccsd0,ccsd1,ccsd2,ccsd)
print(hf_cr+ccsd_cr)

4.405561946234566
-0.009747164169830328
0.030705359520143017 -0.051134426375230684 -0.038552745120460793 -0.05898181197554846
-0.06872897614537879


In [128]:
lno_ccsd._frg_ccsd_orb_cr(prop_data['walkers'][0],ham_data,wave_data,trial)

Array(-0.02517512, dtype=float64)

In [129]:
prop_data,(blk_energy,blk_hf_orb_cr,blk_ccsd_orb_cr,blk_orb_cr,blk_wt)\
    = lno_ccsd.block_orb(prop_data,ham_data,prop,trial,wave_data,sampler)

In [134]:
walker = prop_data['walkers'][0]
hf_cr = lno_ccsd._tot_hf_cr(ham_data['rot_h1'],ham_data['rot_chol'],walker,trial,wave_data)
ccsd_cr0,ccsd_cr1,ccsd_cr2,ccsd_cr = _tot_ccsd_cr(walker,ham_data,wave_data,trial,1e-5)
print(hf_cr)
print(ccsd_cr0,ccsd_cr1,ccsd_cr2,ccsd_cr)
print(hf_cr+ccsd_cr)

4.405561946234566
-0.009747164169830328
0.030705359520143017 -0.051134426375230684 -0.038552745120460793 -0.05898181197554846
-0.06872897614537879


In [131]:
e0, e1_0, e2_0, e_mf, ecorr1, ecorr2, ecorr \
    = _afqmc_ccsd_corr(walker,ham_data,wave_data,trial)
print(e0, e1_0, e2_0, e_mf)
print(ecorr1, ecorr2, ecorr)
print(ecorr+e_mf)

2.29310124732 -6.788713518225568 2.373358607505927 -2.1222536633996407
-0.03980199284071267 -0.05042937664142065 -0.09023136948213331
-2.2124850328817742


In [135]:
print(mf2.e_tot)

-2.112460698914566


In [26]:
# AFQMC energy: -2.18098 +/- 0.00002
# lno tot_afqmc_cr -0.068465 +/- 0.000043
# non-df mean-field energy is -2.112460698914565
-2.18098- -2.112460698914565

-0.06851930108543502